## Covid-19 Hospital Treatment Plan
 #### Nikhil Sai Kumar Vuppalavanchu

 

## Introduction
 We chose this dataset from kaggle: https://www.kaggle.com/datasets/arashnic/covid19-hospital-treatment
 
#### This data is collected from different hospitals of different cities. It consists of 17 independent features such as typeofadmission, hopsital_id etc.. and it has 11 mutliclasses in target variable (stay_days).We have to predict the how long a patient stays in the hospital which helps the management to increase their efficiency in treating the patients.

## Importing all required libraries

In [82]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
#Sampling
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler

#Models
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from xgboost import XGBClassifier
#Metrics
from sklearn.metrics import  confusion_matrix, accuracy_score, precision_score, recall_score, f1_score, classification_report
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, RandomizedSearchCV
from sklearn import preprocessing
#Preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import LabelEncoder

#Plots
from sklearn.tree import plot_tree
from sklearn.tree import export_text
from sklearn.utils import resample

## Loading and preprocessing the data

In [84]:
random_seed = 1
np.random.seed(random_seed)

In [85]:
df = pd.read_csv("host.csv")
print(df.head())
print(df.describe())
print(df.columns)
print(df.dtypes)

   case_id  Hospital  Hospital_type  Hospital_city  Hospital_region  \
0        1         8              2              3                2   
1        2         2              2              5                2   
2        3        10              4              1                0   
3        4        26              1              2                1   
4        5        26              1              2                1   

   Available_Extra_Rooms_in_Hospital    Department Ward_Type Ward_Facility  \
0                                  3  radiotherapy         R             F   
1                                  2  radiotherapy         S             F   
2                                  2    anesthesia         S             E   
3                                  2  radiotherapy         R             D   
4                                  2  radiotherapy         S             D   

   Bed_Grade  patientid  City_Code_Patient Type of Admission Illness_Severity  \
0        2.0      31397

## Checking and handling  missing values 

In [9]:
df.isnull().sum()


case_id                               0
Hospital                              0
Hospital_type                         0
Hospital_city                         0
Hospital_region                       0
Available_Extra_Rooms_in_Hospital     0
Department                            0
Ward_Type                             0
Ward_Facility                         0
Bed_Grade                             1
patientid                             0
City_Code_Patient                    10
Type of Admission                     0
Illness_Severity                      0
Patient_Visitors                      0
Age                                   0
Admission_Deposit                     0
Stay_Days                             0
dtype: int64

In [94]:
old_df =df
df = df.dropna(axis=0)
df.isnull().sum()


case_id                              0
Hospital                             0
Hospital_type                        0
Hospital_city                        0
Hospital_region                      0
Available_Extra_Rooms_in_Hospital    0
Department                           0
Ward_Type                            0
Ward_Facility                        0
Bed_Grade                            0
patientid                            0
City_Code_Patient                    0
Type of Admission                    0
Illness_Severity                     0
Patient_Visitors                     0
Age                                  0
Admission_Deposit                    0
Stay_Days                            0
dtype: int64

## Label encoding the all categorical variables in dataset

In [92]:
df.columns
df.dtypes
cat = [col for col in df.columns if df[col].dtype == 'object']


In [93]:
encoder = LabelEncoder()
for col in cat:
    df[col] = encoder.fit_transform(df[col])

In [95]:

df.corr()

,case_id,Hospital,Hospital_type,Hospital_city,Hospital_region,Available_Extra_Rooms_in_Hospital,Department,Ward_Type,Ward_Facility,Bed_Grade,patientid,City_Code_Patient,Type of Admission,Illness_Severity,Patient_Visitors,Age,Admission_Deposit,Stay_Days
case_id,1.000000,-0.041162,-0.003180,-0.059288,-0.016478,-0.026059,-0.014113,0.004129,0.008032,0.026089,-0.003214,0.026112,0.095440,0.033009,0.094218,0.036126,0.033585,0.114979
Hospital,-0.041162,1.000000,0.027711,0.150871,-0.141737,0.000247,0.011001,0.092938,-0.187778,-0.003104,-0.025765,-0.035208,-0.052293,0.014384,0.030062,0.081311,0.010775,0.093551
Hospital_type,-0.003180,0.027711,1.000000,-0.019628,0.189893,-0.124317,0.031895,0.206125,-0.200429,0.028330,0.051706,0.077692,0.037306,0.010836,-0.081224,-0.024200,-0.018726,0.040327
Hospital_city,-0.059288,0.150871,-0.019628,1.000000,-0.008345,-0.069703,0.069323,0.195374,-0.095202,0.060799,-0.087882,-0.048141,-0.033314,0.011740,0.018059,0.025867,-0.052304,0.035769
Hospital_region,-0.016478,-0.141737,0.189893,-0.008345,1.000000,-0.027704,0.062060,0.155363,-0.446507,0.012199,0.016628,0.144544,-0.015159,0.019545,-0.041911,-0.028706,0.090692,0.042639
Available_Extra_Rooms_in_Hospital,-0.026059,0.000247,-0.124317,-0.069703,-0.027704,1.000000,0.101133,-0.522496,0.086027,-0.119256,-0.019391,-0.070441,0.026464,-0.019168,0.093794,-0.153024,-0.144926,-0.135019
Department,-0.014113,0.011001,0.031895,0.069323,0.062060,0.101133,1.000000,-0.047280,-0.052780,-0.132312,-0.034675,0.042389,-0.133796,-0.110700,0.019592,0.073935,-0.082804,0.009908
Ward_Type,0.004129,0.092938,0.206125,0.195374,0.155363,-0.522496,-0.047280,1.000000,-0.204720,0.046218,-0.013138,0.029025,-0.006157,0.000043,-0.052100,0.109471,0.069786,0.259998
Ward_Facility,0.008032,-0.187778,-0.200429,-0.095202,-0.446507,0.086027,-0.052780,-0.204720,1.000000,0.131046,-0.018067,-0.153363,0.101021,0.005678,0.031373,-0.053412,-0.024339,-0.108182
Bed_Grade,0.026089,-0.003104,0.028330,0.060799,0.012199,-0.119256,-0.132312,0.046218,0.131046,1.000000,0.062444,-0.071497,0.069086,0.114206,0.043381,0.039442,0.093845,-0.043580


In [31]:
print(pd.unique(df['Stay_Days']))
df['Stay_Days'].value_counts()

[ 0  4  3  1  5  2  7 10  8  6  9]


2     313
1     195
3     190
5     126
0      65
4      49
7      46
8      25
10     24
9       9
6       8
Name: Stay_Days, dtype: int64

## Sampling the data

In [96]:
target = 'Stay_Days'

predictors = list(df.columns)
predictors.remove(target)

X=df[predictors]
y=df[target]


rand = RandomOverSampler(random_state=1)
X_result, y_result = rand.fit_resample(X, y)
y_result.value_counts()

0     309
4     309
3     309
1     309
5     309
2     309
7     309
10    309
8     309
6     309
9     309
Name: Stay_Days, dtype: int64

## Splitting the data 

In [97]:
train_X, valid_X, train_y, valid_y = train_test_split(X,y, test_size=0.3, random_state=1)
train_X
valid_X

,case_id,Hospital,Hospital_type,Hospital_city,Hospital_region,Available_Extra_Rooms_in_Hospital,Department,Ward_Type,Ward_Facility,Bed_Grade,patientid,City_Code_Patient,Type of Admission,Illness_Severity,Patient_Visitors,Age,Admission_Deposit
733,734,10,4,1,0,3,2,3,4,2.0,109050,11.0,1,2,4,3,4631
590,591,26,1,2,1,5,2,1,3,2.0,83674,8.0,0,0,4,4,4681
714,715,25,4,1,0,4,2,2,4,4.0,130669,8.0,1,0,2,8,5277
792,793,28,1,11,0,2,2,2,5,3.0,113448,8.0,1,1,3,5,5119
80,81,15,2,5,2,2,2,3,5,2.0,30632,4.0,1,1,3,4,4382
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
521,522,31,2,3,2,3,2,2,0,3.0,76262,15.0,1,0,2,7,4908
284,285,26,1,2,1,3,2,2,3,1.0,109666,9.0,1,0,2,7,5793
458,459,26,1,2,1,4,2,3,3,3.0,129018,4.0,1,2,6,4,4161
547,548,14,0,1,0,3,1,2,4,4.0,117966,23.0,1,2,2,4,5805


## Normalizing using std scaler

In [98]:
scaler = StandardScaler()
scaler.fit(train_X)
norm_train = scaler.transform(train_X)
norm_test= scaler.transform(valid_X)

## KNN model

In [103]:
knn=KNeighborsClassifier()
_ = knn.fit(train_X, train_y)
y_pred = knn.predict(valid_X)
print('Accuracy using default parameters :',accuracy_score(valid_y, y_pred))

Accuracy using default parameters : 0.23076923076923078


In [68]:
score_measure = 'accuracy'




param_grid = {
     'n_neighbors': list(range(1,round(np.sqrt(len(df))),2)),
    'metric': ['euclidean', 'cosine','manhattan','minkowski'],
    'weights':['uniform','distance']
}
gridSearch = GridSearchCV(KNeighborsClassifier(), param_grid, scoring=score_measure,
                          n_jobs=-1)

gridSearch.fit(norm_train, train_y)
print(score_measure, 'score: ', gridSearch.best_score_)
print('parameters: ', gridSearch.best_params_)

KNNAccuracy = gridSearch.best_score_

accuracy score:  0.35076051015588094
parameters:  {'metric': 'manhattan', 'n_neighbors': 7, 'weights': 'distance'}


In [111]:
confusion_matrix(valid_y, y_pred)

array([[ 4,  3,  9,  6,  0,  0,  0,  0,  0,  0,  0],
       [ 2, 18, 24,  3,  1,  3,  0,  1,  0,  1,  0],
       [ 1, 24, 61, 10,  1,  1,  0,  1,  0,  1,  0],
       [ 1,  9, 22, 13,  1,  9,  0,  2,  0,  0,  0],
       [ 0,  4,  7,  2,  3,  0,  1,  0,  0,  0,  0],
       [ 0,  0,  5,  5,  2, 12,  0,  3,  2,  0,  1],
       [ 0,  0,  1,  1,  0,  0,  1,  0,  0,  0,  0],
       [ 0,  1,  2,  1,  0,  4,  0,  2,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  4,  0,  2,  6,  0,  0],
       [ 0,  0,  0,  1,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  1,  0,  1,  3,  0,  2]], dtype=int64)

## We got accuracy score of 23% with default. After hyperparameter pruning we got accuracy of 35%. We use three parameters namely n_neighbours, meterix, weights
## best metric we got  is manhattan, with neighbours 7

## Decision Tree

In [100]:
dtree=DecisionTreeClassifier(random_state=3)
_ = dtree.fit(train_X, train_y)
y_pred = dtree.predict(valid_X)
print('Accuracy using default parameters :',accuracy_score(valid_y, y_pred))

Accuracy using default parameters : 0.3269230769230769


In [51]:
score_measure = 'accuracy'


param_grid = { 
              'max_depth': [2, 5, 10, 20, 30, 40],
              'min_samples_split': [6, 10,20],       
              'min_samples_leaf': [1, 3,  4],
    'min_impurity_decrease': [0, 0.0005, 0.001, 0.005, 0.01]
              }
gridSearch = GridSearchCV(DecisionTreeClassifier(random_state=1), param_grid, scoring=score_measure,
                          n_jobs=-1)

gridSearch.fit(train_X, train_y)
print(score_measure, 'score: ', gridSearch.best_score_)
print('parameters: ', gridSearch.best_params_)

DTreeAccuracy = gridSearch.best_score_

accuracy score:  0.36859707132735003
parameters:  {'max_depth': 2, 'min_impurity_decrease': 0.01, 'min_samples_leaf': 1, 'min_samples_split': 6}


## We got the accuracy of 32.6% with default decision tree and after parameter tuning with max-depth. min-samples-split, min-sample-leaf, min-impurity-decrease accuracy is 36.8% 
## We got max_depth as 2, min_samples_leaf as 1,min_samples_split as.
## Decision tree is better model than KNN

## Random Forest

In [104]:
forest = RandomForestClassifier(random_state=1)
_ = rforest.fit(train_X, train_y)
y_pred = rforest.predict(valid_X)
print('Accuracy using default parameters :',accuracy_score(valid_y, y_pred))

Accuracy using default parameters : 0.3814102564102564


In [81]:
score_measure = 'accuracy'


rforest = RandomForestClassifier(random_state=random_seed)
_ = rforest.fit(train_X, train_y)

y_pred = rforest.predict(valid_X)

param_grid = {
      'n_estimators': range(40,100,5),
    'max_features':np.arange(0.1, 1, 0.1),
    'max_depth': [3, 5, 7, 9],
    'max_samples': [0.3, 0.5, 0.8], 

         
            
}
gridSearch = GridSearchCV(RandomForestClassifier(random_state=random_seed), param_grid, scoring=score_measure,
                          n_jobs=-1)

gridSearch.fit(train_X, train_y)
print(score_measure, 'score: ', gridSearch.best_score_)
print('parameters: ', gridSearch.best_params_)

RandomForestAccuracy = gridSearch.best_score_

accuracy score:  0.407094945677846
parameters:  {'max_depth': 9, 'max_features': 0.5, 'max_samples': 0.5, 'n_estimators': 70}


## We got the accuracy of 38.1% with default random forest and after parameter tuning with max-depthl,n_estimators,max_samples accuracy is 40.7% 
## We got max_depth as 9, n_estimators as 70,max_samples as 0.5
## We got Random forest is the best model of accuracy with 40.7%

## AdaBoostClassifier

In [106]:
abo = AdaBoostClassifier(random_state=1)
_ = aboost.fit(train_X, train_y)
y_pred = aboost.predict(valid_X)
print('Accuracy using default parameters :',accuracy_score(valid_y, y_pred))

Accuracy using default parameters : 0.3141025641025641


In [80]:
score_measure = 'accuracy'


aboost = AdaBoostClassifier(random_state=random_seed)
_ = aboost.fit(train_X, train_y)
y_pred = aboost.predict(valid_X)


param_grid = { 
              'n_estimators':range(1,50,1),
    'learning_rate': [0.5,0.7,0.8,0.9,1.0]
              }

gridSearch = GridSearchCV(AdaBoostClassifier(random_state=random_seed), param_grid, scoring=score_measure,
                          n_jobs=-1)
gridSearch.fit(train_X, train_y)
print(score_measure, 'score: ', gridSearch.best_score_)
print('parameters: ', gridSearch.best_params_)

AdaAccuracy = gridSearch.best_score_

accuracy score:  0.3328483703353803
parameters:  {'learning_rate': 0.9, 'n_estimators': 4}


## We got the accuracy of 31.4% with default AdaBoost and after parameter tuning with learning_rate,n_estimators accuracy is 33.28% 
## We got n_estimators as 4,learning_rate as 0.9


## XGBoost Classifier

xgb = XGBClassifier(random_state=1)
_ = xgb.fit(train_X, train_y)
y_pred = xgb.predict(valid_X)
print('Accuracy using default parameters :',accuracy_score(valid_y, y_pred))

In [73]:
score_measure = 'accuracy'


xgboost = XGBClassifier(random_state=random_seed)
_ = xgboost.fit(train_X, train_y)
y_pred = xgboost.predict(valid_X)


param_grid = {
    'max_depth': range(2,10,1),
           'learning_rate': [0.1, 0.01, 0.05],
           'n_estimators': range(40,220,40)
              
}
gridSearch = GridSearchCV(XGBClassifier(random_state=random_seed), param_grid, scoring=score_measure,
                          n_jobs=-1)
gridSearch.fit(train_X, train_y)
print(score_measure, 'score: ', gridSearch.best_score_)
print('parameters: ', gridSearch.best_params_)

XGBoostAccuracy = gridSearch.best_score_

C:\Users\18138\anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
C:\Users\18138\anaconda3\lib\site-packages\xgboost\data.py:250: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


[21:57:37] WARNING: C:\Windows\Temp\abs_557yfx631l\croots\recipe\xgboost-split_1659548953302\work\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


C:\Users\18138\anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
C:\Users\18138\anaconda3\lib\site-packages\xgboost\data.py:250: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


[22:02:10] WARNING: C:\Windows\Temp\abs_557yfx631l\croots\recipe\xgboost-split_1659548953302\work\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
accuracy score:  0.38653755314123756
parameters:  {'learning_rate': 0.05, 'max_depth': 8, 'n_estimators': 200}


## We got the accuracy of 38.4% with default XGBoost and after parameter tuning with learning_rate,n_estimators accuracy is 38.65% 
## We got n_estimators as 200,learning_rate as 0.05


## Gradient Boost Classifier

In [110]:
score_measure = 'accuracy'


gboost = GradientBoostingClassifier(random_state=random_seed)
_ = gboost.fit(train_X, train_y)
y_pred = gboost.predict(valid_X)


param_grid = {
    'n_estimators': range(40,200,10),
    'learning_rate': (0.1,0.2,0.3,0.4,0.5)
}
gridSearch = GridSearchCV(GradientBoostingClassifier(random_state=random_seed), param_grid, scoring=score_measure,
                          n_jobs=-1)
gridSearch.fit(train_X, train_y)
print(score_measure, 'score: ', gridSearch.best_score_)
print('parameters: ', gridSearch.best_params_)

GBoostAccuracy = gridSearch.best_score_

accuracy score:  0.36312706660368443
parameters:  {'learning_rate': 0.3, 'n_estimators': 150}


## The most accuracy is 40% which we got with RandomForest. Using RandomForest model, we can predict the accurate results of number of days going to stay in hospital to increase the efficency in treatment.
## As it is taking longer time for executing, we took some randoms rows of data, this may be the cause of getting lower numbers for accuracy.
## We choose best metric here is Accuracy, we got higher accuracy value when compared to recall and precision and also we got more true positive and true negative values.